In [1]:
import sdm as sdmlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
%matplotlib inline

In [2]:
bits = 1000
sample = 1000000
scanner_type = sdmlib.SDM_SCANNER_OPENCL

In [3]:
address_space = sdmlib.AddressSpace.init_random(bits, sample)
counter = sdmlib.Counter.create_file('sdm-tmp-DELETE-IT', bits, sample)
sdm = sdmlib.SDM(address_space, counter, 451, scanner_type)

In [4]:
def write(n=1000, prefix=''):
    for i in xrange(n):
        clear_output(wait=True)
        print '{}Writing {} random bitstrings: {:4d} ({:.2f}%)'.format(prefix, n, i+1, 100.*(i+1)/n)
        b = sdmlib.Bitstring.init_random(bits)
        sdm.write(b, b)
    return n

In [5]:
def calculate_distances(bs, k=6, step=1, prefix=''):
    distances = []
    x = range(0, bits+1, step)
    for i, dist in enumerate(x):
        clear_output(wait=True)
        print '{}Distance: {:4d} ({:.2f}%)'.format(prefix, dist, 100.*(i+1)/len(x))
        c = sdmlib.Bitstring.init_from_bitstring(bs)
        c.flip_random_bits(dist)
        assert c.distance_to(bs) == dist
        v = []
        v.append(c.distance_to(bs))
        d = c
        for j in xrange(k):
            d = sdm.read(d)
            v.append(d.distance_to(bs))
        distances.append([dist, v])
    return distances

In [6]:
bs_ref = sdmlib.Bitstring.init_random(bits)
sdm.write(bs_ref, bs_ref)

In [ ]:
distances = []
qty = 1000
n = 50
step = 1
k = 6
total = 0
for i in xrange(n):
    prefix = '[{}] '.format(total)
    total += write(qty, prefix=prefix)
    v = calculate_distances(bs_ref, k=k, step=step, prefix=prefix)
    distances.append((total, v))

[0] Writing 1000 random bitstrings:  245 (24.50%)


In [ ]:
def plot_heatmap(data):
    # Make plot with vertical (default) colorbar
    maxd = int(data.max())
    mind = int(data.min())
    avgd = int ((maxd+mind)/2);
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    #use aspect=20 when N=1000
    #use aspect=5 when N=256
    
    from matplotlib.pylab import cm, contourf, contour
    
    cax = ax.imshow(data, cmap=cm.YlGnBu, aspect='auto', interpolation=None, norm=None, origin='lower')
    ax.grid(True, label='Distance')

    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    cbar = fig.colorbar(cax, ticks=[mind, avgd, maxd]) #had ZERO here before
    cbar.ax.set_yticklabels([str(mind), str(avgd), str(maxd)])
    cbar.ax.set_ylabel('Distance obtained after X iteractive-readings', fontsize=12)
        
    #########CONTOUR DELINEATES THE CRITICAL DISTANCE

    # We are using automatic selection of contour levels;
    # this is usually not such a good idea, because they don't
    # occur on nice boundaries, but we do it here for purposes
    # of illustration.
    CS = ax.contourf(data, 100, levels=[mind,avgd,maxd], alpha=0.1, cmap=cm.YlGnBu, origin='lower')
    
    # Note that in the following, we explicitly pass in a subset of
    # the contour levels used for the filled contours.  Alternatively,
    # We could pass in additional levels to provide extra resolution,
    # or leave out the levels kwarg to use all of the original levels.
    CS2 = ax.contour(CS,  levels=[88], colors='gray', origin='lower', hold='on', linestyles='dashdot')
    
    plt.title('Critical Distance Behavior', fontsize=20)
    plt.xlabel('Original distance', fontsize=12)
    plt.ylabel('# items previously stored (000\'s)', fontsize=12)
    # Add the contour line levels to the colorbar
    #cbar.add_lines(CS2)
    plt.show()

In [ ]:
def get_data(distances, k):
    # k = iterative readings
    
    # [(x, y, z)]
    # x = old distance
    # y = number of writings
    # z = new distance
    data = []
    for writings, v1 in distances:
        v = []
        for old, v2 in v1:
            new = v2[k]
            #data.append([old, writings, new])
            v.append(new)
        data.append(v)
    return np.array(data)

In [ ]:
data = get_data(distances, 0)
plot_heatmap(data)
#print len(data), len(data[0])